In [ ]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, Conv2D,GlobalAvgPool2D, Input
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras import callbacks, optimizers
import numpy as np
from google.colab import drive
import tensorflow_datasets as tfds #imported to get the plant_village dataset
import matplotlib.pyplot as plt
import random

from tensorflow.keras import layers
from sklearn.model_selection import train_test_split #split the data
from tensorflow.keras.models import Sequential

drive.mount('/content/drive/')

%cd drive/MyDrive/

#listing the classes in the dataset
for i in os.listdir("Plant_Dataset"):
  print(i, len(os.listdir("Plant_Dataset/"+i)))



In [ ]:
# Create DL_dataset folder with train, test, and validate subfolders
#base_folder = "DL_Dataset3"
#for subfolder in ["train", "test", "validate"]:
    #os.makedirs(os.path.join(base_folder, subfolder), exist_ok=True)
    #print(os.path.join(base_folder, subfolder))

# Loop through the dataset in the Plant_Dataset folder and split it
#for class_folder in os.listdir("Plant_Dataset"):
    # Create subdirectories for each class in train, test, and validate folders
    #os.makedirs(os.path.join(base_folder, "train", class_folder), exist_ok=True)
    #os.makedirs(os.path.join(base_folder, "test", class_folder), exist_ok=True)
    #os.makedirs(os.path.join(base_folder, "validate", class_folder), exist_ok=True)

    # Move images to train, test, and validate folders
    #images = os.listdir(os.path.join("Plant_Dataset", class_folder))
    #for img in images[:600]:
        #shutil.copy(os.path.join("Plant_Dataset", class_folder, img), os.path.join(base_folder, "train", class_folder, img))
    #for img in images[600:850]:
        #shutil.copy(os.path.join("Plant_Dataset", class_folder, img), os.path.join(base_folder, "test", class_folder, img))
    #for img in images[850:1000]:
        #shutil.copy(os.path.join("Plant_Dataset", class_folder, img), os.path.join(base_folder, "validate", class_folder, img))

In [ ]:
# Function to create ImageDataGenerator for train, test, and validate datasets
def img_data(dir_path, target_size, batch_size, class_lst, preprocess_func=None):
    gen_object = ImageDataGenerator(preprocessing_function=preprocess_func) if preprocess_func else ImageDataGenerator()
    return gen_object.flow_from_directory(
        dir_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='sparse',
        classes=class_lst,
        shuffle=True
    )

# Define class list and initialize data generators using DL_dataset folder
base_folder = "DL_Dataset3"
class_list = os.listdir(os.path.join(base_folder, "train"))

train_data_gen = img_data(os.path.join(base_folder, "train"), (224, 224), 500, class_list, preprocess_input)
test_data_gen = img_data(os.path.join(base_folder, "test"), (224, 224), 500, class_list, preprocess_input)
valid_data_gen = img_data(os.path.join(base_folder, "validate"), (224, 224), 500, class_list, preprocess_input)

# Load and set up MobileNetV2 model
my_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(224, 224, 3),
    alpha=1.0,
    include_top=False,
    weights='imagenet',
    pooling=None,
    classifier_activation='softmax'
)
my_model.trainable = False

# Build and compile model
model = tf.keras.models.Sequential([
    my_model,
    GlobalAvgPool2D(),
    Dense(1024, activation='relu'),
    Dense(len(class_list), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define callbacks and train the model
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')
save_ckpt = callbacks.ModelCheckpoint(".myModel2.keras", save_best_only=True, monitor='val_loss', mode='min')

# Fit model
model.fit(
    train_data_gen,
    validation_data=valid_data_gen,
    epochs=10,
    callbacks=[early_stop, save_ckpt]
)

print('model saved')

Found 22800 images belonging to 38 classes.
Found 9500 images belonging to 38 classes.
Found 5700 images belonging to 38 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


46/46 ━━━━━━━━━━━━━━━━━━━━ 8664s 167s/step - accuracy: 0.6392 - loss: 1.4572 - val_accuracy: 0.8937 - val_loss: 0.3436
Epoch 2/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1588s 34s/step - accuracy: 0.9334 - loss: 0.2134 - val_accuracy: 0.9188 - val_loss: 0.2708
Epoch 3/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1638s 34s/step - accuracy: 0.9580 - loss: 0.1400 - val_accuracy: 0.9139 - val_loss: 0.2840
Epoch 4/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1601s 34s/step - accuracy: 0.9730 - loss: 0.1031 - val_accuracy: 0.9216 - val_loss: 0.2475
Epoch 5/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1639s 34s/step - accuracy: 0.9768 - loss: 0.0834 - val_accuracy: 0.9146 - val_loss: 0.2974
Epoch 6/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1652s 34s/step - accuracy: 0.9859 - loss: 0.0617 - val_accuracy: 0.9367 - val_loss: 0.2010
Epoch 7/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1581s 34s/step - accuracy: 0.9912 - loss: 0.0447 - val_accuracy: 0.9242 - val_loss: 0.2541
Epoch 8/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1578s 34s/step - accuracy: 0.9875 - loss: 0.0471 - val_accuracy: 0.93